In [2]:
##Estimación de indicadores de agua
#Se borra todo lo que se encuentra en el entorno

#rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr)

Loading required package: pacman



In [3]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [4]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/cngmd/"
fin<-"_csv.zip"

In [7]:
#Loop para descargar los archivos
for (i in 1:length(years)) {
  # Condición según el año
  if (years[i] == 2019) {
    segunda_parte <- "datosabiertos/m5/capta_agua_cngmd"
  }
  else if (years[i] == 2021) {
    segunda_parte <- "datosabiertos/m5/captacion_agua_abastecimiento_pub_cngmd"
  }
  else {
    segunda_parte <- "datosabiertos/m6/captacion_agua_abastecimiento_pub_cngmd"
  }

  # url
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  print(url)

  # Destino del archivo
  destfile <- paste0("data/", years[i], fin)

  # Descarga
  download.file(url, destfile = destfile, mode = "wb")

  # Unzip
  unzip(destfile, exdir = "data")
  unlink(destfile)
}

[1] "https://www.inegi.org.mx/contenidos/programas/cngmd/2019/datosabiertos/m5/capta_agua_cngmd2019_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/cngmd/2021/datosabiertos/m5/captacion_agua_abastecimiento_pub_cngmd2021_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/cngmd/2023/datosabiertos/m6/captacion_agua_abastecimiento_pub_cngmd2023_csv.zip"


In [17]:
# Función de estimación
process_files <- function(years, variable) {
  # Función anidada para estimar el indicador
  estima_indicador <- function(path, variable) {
    # Extraer el año del archivo
    year <- substr(path, 11, 14)

    # Leer los datos y hacer las transformaciones necesarias de las variables
    if (year %in% c("2017", "2019")) {
      datos <- read_csv(path, show_col_types = FALSE) %>%
        janitor::clean_names() %>%
        #generar clave de entidad
    mutate(cve_ent = substr(cve_geo, 1, 2))%>%
    filter(cve_ent == "09")


    } else if (year %in% c("2021", "2023")) {
        datos <- read_csv(path, show_col_types = FALSE) %>%
        janitor::clean_names() %>%
        #generar clave de entidad
    mutate(cve_ent = substr(cve_geo, 1, 2))%>%
    filter(cve_ent == "09")
    } else {
      stop("Año no soportado.")
    }

  }

    # Macromedidor
    if (variable == "macromedidor") {
      macromedidor <- datos %>%
        mutate(funciona=case_when(
            fnt_mcro == 2 | fnt_mcro == 3 ~ "No funciona",
            TRUE ~ "Funciona"
        )) %>%
        group_by(funciona) %>%
        mutate(
        #Total de obras
        obras = n(),
        #Obras que no funcionan
        porcentaje = round(n()/nrow(datos)*100,2)%>%
        ungroup()%>%
        select(funciona, obras, porcentaje)%>%
        mutate(year = year)
        return(macromedidor)
}



  map_dfr(years, function(year) {
    # Seleccionar el archivo correcto según la variable y el año
    file_path <- if (variable == "macromedidor") {
      if (year %in% c(2017, 2019)) {
        paste0("data/encig", year, "_01_sec1_3_4_5_8_9_10.dbf")
      } else if (year %in% c(2021, 2023)) {
        paste0("data/conjunto_de_datos/fntcapta_cngmd",year,".csv")
      } else {
        stop("Año no soportado.")
      }
    } else {
      stop("Variable no reconocida.")
    }

    # Print del archivo que se está procesando
    print(paste("Procesando archivo:", file_path))

    # Invocar la función de estimación
    estima_indicador(file_path, variable)
  })
}


ERROR: Error in parse(text = x, srcfile = src): <text>:49:9: unexpected symbol
48:         mutate(year = year)
49:         return
            ^


In [51]:
# Función de estimación
process_files <- function(years, variable) {

  # Función anidada para estimar el indicador
  estima_indicador <- function(path, variable, year) {
    # Leer los datos y hacer las transformaciones necesarias de las variables
    if (year %in% c("2017", "2019")) {
      datos <- read_csv(path, show_col_types = FALSE) %>%
        janitor::clean_names() %>%
        mutate(cve_ent = substr(folio, 1, 2)) %>%
        filter(cve_ent == "09")
    } else if (year %in% c("2021", "2023")) {
      datos <- read_csv(path, show_col_types = FALSE) %>%
        janitor::clean_names() %>%
        mutate(cve_ent = substr(cve_geo, 1, 2)) %>%
        filter(cve_ent == "09")
    } else {
      stop("Año no soportado.")
    }

    # Procesar la variable macromedidor
    if (variable == "macromedidor") {
      macromedidor <- datos %>%
      mutate(fnt_mcro = if ("fntmacro" %in% names(.)) fntmacro else fnt_mcro) %>%


        mutate(funciona = case_when(
          fnt_mcro == 2 | fnt_mcro == 3 ~ "No funciona",
          TRUE ~ "Funciona"
        )) %>%
        group_by(funciona) %>%
        summarise(
          obras = n(),
          porcentaje = round(n() / nrow(datos) * 100, 2)
        ) %>%
        ungroup() %>%
        mutate(year = year)%>%
        filter(funciona == "No funciona")

      return(macromedidor)
    } else {
      stop("Variable no reconocida.")
    }
  }

  # Procesar cada año y consolidar los resultados
  results <- map_dfr(years, function(year) {
    # Seleccionar el archivo correcto según la variable y el año
    file_path <- if (variable == "macromedidor") {
      if (year %in% c(2017, 2019)) {
        paste0("data/capta_agua_cngmd", year, "_csv/conjunto_de_datos/secc_ii_td_m5_", (year - 1), "_funt_capt.csv")
      } else if (year %in% c(2021, 2023)) {
        paste0("data/conjunto_de_datos/fntcapta_cngmd", year, ".csv")
      } else {
        stop("Año no soportado.")
      }
    } else {
      stop("Variable no reconocida.")
    }

    # Print del archivo que se está procesando
    print(paste("Procesando archivo:", file_path))

    # Invocar la función de estimación y devolver el resultado
    estima_indicador(file_path, variable, year)
  })

  return(results)
}

In [52]:
process_files(c(2019, 2021, 2023), "macromedidor")

[1] "Procesando archivo: data/capta_agua_cngmd2019_csv/conjunto_de_datos/secc_ii_td_m5_2018_funt_capt.csv"
[1] "Procesando archivo: data/conjunto_de_datos/fntcapta_cngmd2021.csv"
[1] "Procesando archivo: data/conjunto_de_datos/fntcapta_cngmd2023.csv"


funciona,obras,porcentaje,year
<chr>,<int>,<dbl>,<dbl>
No funciona,11,1.96,2019
No funciona,82,15.95,2021
No funciona,81,15.61,2023
